## A driver can only take one order at a time

In [1]:
import pickle

import numpy as np
import pandas as pd

from functools import partial
from geopy.distance import geodesic

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:
path = "input_data"

READ INPUT DATA FILES

In [3]:
order_df = pd.read_csv(f"{path}/order_data.csv")
order_df.head()

,order_id,restaurant_lat,restaurant_long,customer_lat,customer_long,no_of_items,prep_duration_sec,preferred_otd_sec
0,0,52.534717,13.392920,52.536950,13.363647,9,960,2400
1,1,52.507490,13.460169,52.521861,13.464337,10,780,2400
2,2,52.539029,13.384884,52.507035,13.459327,12,360,2400
3,3,52.508368,13.408875,52.519709,13.374291,16,1020,2400
4,4,52.537509,13.429478,52.544824,13.432814,15,360,2400


In [4]:
driver_df = pd.read_csv(f"{path}/driver_data.csv")
driver_df.head()

,driver_id,shift_start_sec,shift_end_sec,start_location_lat,start_location_long,vehicle
0,0,0,7200,52.502670,13.447406,BIKE
1,1,0,7200,52.518258,13.447747,BIKE
2,2,0,7200,52.517876,13.460788,BIKE
3,3,0,7200,52.528275,13.431436,BIKE
4,4,0,7200,52.510854,13.454662,BIKE


In [5]:
# Check initial sanity of data
order_df.describe()

,order_id,restaurant_lat,restaurant_long,customer_lat,customer_long,no_of_items,prep_duration_sec,preferred_otd_sec
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0
mean,49.500000,52.520640,13.408850,52.518894,13.406724,9.840000,723.600000,2400.0
std,29.011492,0.017904,0.028543,0.019089,0.028775,5.052692,267.489337,0.0
min,0.000000,52.485165,13.353377,52.486500,13.354016,1.000000,300.000000,2400.0
25%,24.750000,52.507638,13.388908,52.502885,13.384448,6.000000,480.000000,2400.0
50%,49.500000,52.519565,13.407955,52.518842,13.401845,9.000000,720.000000,2400.0
75%,74.250000,52.534732,13.429163,52.534923,13.430591,14.250000,960.000000,2400.0
max,99.000000,52.554721,13.464321,52.555948,13.464337,19.000000,1200.000000,2400.0


In [6]:
driver_df.describe()

,driver_id,shift_start_sec,shift_end_sec,start_location_lat,start_location_long
count,40.000000,40.0,40.0,40.000000,40.000000
mean,19.500000,0.0,7200.0,52.520005,13.406796
std,11.690452,0.0,0.0,0.015724,0.030908
min,0.000000,0.0,7200.0,52.488881,13.351823
25%,9.750000,0.0,7200.0,52.509868,13.381836
50%,19.500000,0.0,7200.0,52.519269,13.410317
75%,29.250000,0.0,7200.0,52.530530,13.422828
max,39.000000,0.0,7200.0,52.555571,13.460788


In [7]:
# We have used geodesics to calculate distances between two coordinates
def calculate_distance(coord1, coord2):
    return int(geodesic(coord1, coord2).meters)

In [8]:
pickup_coordinates = order_df[["restaurant_lat", "restaurant_long"]].values.tolist()
drop_coordinates = order_df[["customer_lat", "customer_long"]].values.tolist()

In [9]:
veh_start_coordinates = driver_df[["start_location_lat", "start_location_long"]].values.tolist()

In [10]:
cust_coordinates = pickup_coordinates + drop_coordinates

In [11]:
len(pickup_coordinates)

100

In [12]:
len(cust_coordinates)

200

In [13]:
all_coordinates = cust_coordinates + veh_start_coordinates
all_coordinates_count = len(all_coordinates)
all_coordinates_count

240

In [14]:
# 1 dummy node at beginning to allow arbitrary end locations of routes
distance_matrix = [[0] * (all_coordinates_count + 1) for _ in range(all_coordinates_count + 1)]

In [15]:
len(distance_matrix)

241

In [16]:
len(distance_matrix[0])

241

In [17]:
# for i in range(all_coordinates_count):
#     for j in range(all_coordinates_count):
#         distance_matrix[i+1][j+1] = calculate_distance(all_coordinates[i], all_coordinates[j])

In [18]:
# with open("dm", "wb") as fp:   #Pickling
#   pickle.dump(distance_matrix, fp)

In [19]:
with open("dm", "rb") as fp:   # Unpickling
  distance_matrix = pickle.load(fp)

In [20]:
data = {}
data["distance_matrix"] = distance_matrix
data["num_vehicles"] = driver_df.shape[0]

In [21]:
data["veh_start_indices"] = [len(cust_coordinates) + i + 1 for i in range(driver_df.shape[0])]

In [22]:
data["veh_end_indices"] = [0 for _ in range(driver_df.shape[0])]

In [23]:
data["num_vehicles"]

40

In [24]:
len(data["distance_matrix"])

241

In [25]:
driver_df["cost_per_meter"] = driver_df.vehicle.apply(lambda x: 1 if x=="BIKE" else 3)
driver_df["speed_meter_per_sec"] = driver_df.vehicle.apply(lambda x: 15*1000/3600 if x=="BIKE" else 30*1000/3600)

In [26]:
# Create the Routing Index Manager
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]), data["num_vehicles"], data["veh_start_indices"], data["veh_end_indices"]
)

In [27]:
# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

In [28]:
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index) 

    # Calculate distance between nodes 
    distance = data["distance_matrix"][from_node][to_node]

    return distance


def cost_callback(from_index, to_index, cost_per_meter):

    distance = distance_callback(from_index, to_index)
    cost = distance * cost_per_meter

    return cost

In [29]:
bike_transit_cost_callback = partial(cost_callback, cost_per_meter=1)
car_transit_cost_callback = partial(cost_callback, cost_per_meter=3)

In [30]:
distance_callback_index = routing.RegisterTransitCallback(distance_callback)

bike_transit_cost_callback_index = routing.RegisterTransitCallback(bike_transit_cost_callback)
car_transit_cost_callback_index = routing.RegisterTransitCallback(car_transit_cost_callback)

In [31]:
# Define cost of each arc.
for _, row in  driver_df.iterrows():
    v_id = row["driver_id"]
    v_type = row["vehicle"]
    cost = row["cost_per_meter"]
    if v_type == "BIKE":
        routing.SetArcCostEvaluatorOfVehicle(bike_transit_cost_callback_index, v_id)
    elif v_type == "CAR":
        routing.SetArcCostEvaluatorOfVehicle(car_transit_cost_callback_index, v_id)

In [32]:
# Add Distance dimension to track distances
dimension_name = "Distance"
routing.AddDimension(
    distance_callback_index,
    0,  # no slack
    int(sum(data["distance_matrix"][1])),  # vehicle maximum travel distance to be large
    True,  # start cumul to zero
    dimension_name,
)
distance_dimension = routing.GetDimensionOrDie(dimension_name)

In [33]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

In [34]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [35]:
print(f"Objective Cost: ${solution.ObjectiveValue()}")
max_route_cost = 0
for vehicle_id in range(data["num_vehicles"]):
    index = routing.Start(vehicle_id)
    plan_output = f"Route for vehicle {vehicle_id}:\n"
    route_cost = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} -> "
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_cost += routing.GetArcCostForVehicle(
            previous_index, index, vehicle_id
        )
    plan_output += f"{manager.IndexToNode(index)}\n"
    plan_output += f"Cost of the route: ${route_cost}\n"
    print(plan_output)
    max_route_cost = max(route_cost, max_route_cost)
print(f"Maximum of the route costs: ${max_route_cost}")

Objective Cost: $67038
Route for vehicle 0:
 201 ->  130 ->  129 ->  115 ->  89 ->  182 ->  138 ->  136 ->  190 ->  103 ->  2 -> 0
Cost of the route: $2816

Route for vehicle 1:
 202 ->  22 ->  14 ->  131 ->  200 -> 0
Cost of the route: $1010

Route for vehicle 2:
 203 ->  191 ->  45 ->  102 -> 0
Cost of the route: $574

Route for vehicle 3:
 204 ->  66 ->  171 ->  181 ->  5 ->  105 ->  60 ->  107 ->  23 ->  140 ->  65 ->  68 -> 0
Cost of the route: $4468

Route for vehicle 4:
 205 ->  16 -> 0
Cost of the route: $281

Route for vehicle 5:
 206 ->  153 ->  121 ->  6 ->  125 ->  172 ->  31 ->  53 ->  24 ->  12 ->  20 ->  156 ->  127 ->  187 ->  33 ->  170 ->  174 ->  141 ->  162 ->  133 ->  123 ->  155 ->  30 ->  86 ->  137 ->  4 ->  38 ->  119 ->  145 ->  39 ->  177 ->  164 ->  118 ->  44 ->  100 ->  166 ->  94 ->  148 ->  93 ->  76 ->  196 ->  58 ->  188 ->  168 ->  184 ->  36 ->  112 ->  132 ->  57 ->  144 ->  175 ->  198 ->  35 ->  122 ->  71 -> 0
Cost of the route: $18321

Route for

In [36]:
(60374, 7788) , (67038, 18321)

((60374, 7788), (67038, 18321))